In [1]:
import pickle
import os
import torch
import numpy as np
from util.evaluate import Evaluator
from sklearn import metrics
from sklearn.metrics import auc

In [2]:
device = "cuda:0"

In [3]:
from factory.MetaDV import MetaDV
C = MetaDV(80,256).to(device)
C.load_state_dict(torch.load("model/static/metadv_vctk80.pth",map_location=device))

<All keys matched successfully>

In [4]:
import json
json_config = json.load(open("config.json"))

In [5]:
class Config:
    def __init__(self,root):
        self.root = root
        self.device = device
        self.embedder = C
        self.metadata = pickle.load(open(f'{root}/train.pkl', "rb"))
        self.num_speaker = json_config["eval_speaker"]
        self.batch_size = json_config["batch_size"]
        self.erroment_num = json_config["erroment_num"]
        self.max_uttr_idx = json_config["num_uttr"] - 1
        self.len_crop = json_config["len_crop"]
        self.pick_speaker = json_config["pick_speaker"]

In [7]:
config = Config('train_spmel_vctk80')

In [8]:
all_real_cos_result = []
all_real_var_result = [] 
all_non_real_cos_result = []
all_non_real_var_result = []
all_yt, all_ys = [],[]

for i in range(20):
    E = Evaluator(config,seed=i)
    real_cos_result, real_var_result = E.get_real_data_cos()
    y_true, y_score = E.get_ytrue_yscore([real_cos_result])
    for ele in y_true:
        all_yt.append(ele)
    for ele in y_score:
        all_ys.append(ele)
    ## Mean of 40 speaker Cos-similarity
    all_real_cos_result.append(np.diagonal(real_cos_result).mean())
    ## Mean of 40 speaker Cos-similarity Variance
    all_real_var_result.append(np.diagonal(real_var_result).mean())
    ##  Mean of 40 speaker Cos-similarity with non_source speaker embedding
    all_non_real_cos_result.append((np.sum(real_cos_result) - (np.sum(np.diagonal(real_cos_result)))) /(config.num_speaker*(config.num_speaker-1)))
    ##  Mean of 40 speaker Cos-similarity Variance with non_source speaker embedding
    all_non_real_var_result.append((np.sum(real_var_result) - (np.sum(np.diagonal(real_var_result)))) /(config.num_speaker*(config.num_speaker-1)))
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)

--- p227 in metadata ---
--- p228 in metadata ---
--- p229 in metadata ---
Detect Embedder ! generate all Real Data d-vector
Processing --- ID:0 Speaker:p227 ---
Processing --- ID:1 Speaker:p228 ---
Processing --- ID:2 Speaker:p229 ---
Processing --- ID:0 Speaker:p227 ---
Processing --- ID:1 Speaker:p228 ---
Processing --- ID:2 Speaker:p229 ---


In [9]:
np.array(all_real_cos_result).mean(),np.array(all_real_cos_result).std()

(0.8173573017120361, 0.0)

In [10]:
np.array(all_real_var_result).mean(),np.array(all_real_var_result).std()

(0.09198160221179326, 0.0)

In [11]:
np.array(all_non_real_cos_result).mean(),np.array(all_non_real_cos_result).std()

(0.08181153719609331, 0.0)

In [12]:
np.array(all_non_real_var_result).mean(),np.array(all_non_real_var_result).std()

(0.038208672194741666, 0.0)

In [13]:
import csv
with open(f"realdata_result.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(tpr)
    writer.writerow(fpr)
    writer.writerow(thresholds)
    writer.writerow([auc(fpr, tpr),np.array(all_real_cos_result).mean(),np.array(all_real_cos_result).std(),np.array(all_non_real_cos_result).mean(),np.array(all_non_real_cos_result).std()])